In [ ]:

gmx_mpi pdb2gmx -f ../template.pdb -o template.gro -water tip3p

# -------------------------------------------------------
# Program:     gmx pdb2gmx, version 2022.5-plumed_2.9.0
# Source file: src/gromacs/gmxpreprocess/ter_db.cpp (line 138)
# Function:    void read_atom(char*, bool, std::string*, t_atom*, PreprocessingAtomTypes*, int*)

# Inconsistency in user input:
# Atom type OH specified in terminal database has not been defined in the force
# field

# For more information and tips for troubleshooting, please check the GROMACS
# website at http://www.gromacs.org/Documentation/Errors
# -------------------------------------------------------

# Changed aminoacids.c.tdb OH to OH1.This worked! Potential typo in force field files?

gmx_mpi editconf -f ../template.gro -o template_newbox.gro -c -d 1.0 -bt dodecahedron
# output of the above command Volume: 362.038 nm^3

gmx_mpi editconf -f ../template.gro -o template_newbox2.gro -c -d 2.0 -bt dodecahedron

gmx_mpi solvate -cp template_newbox.gro -cs spc216.gro -o template_solv.gro -p ../topol.top
# only 5922 solvent molecules added compared to 11000 in the paper

gmx_mpi solvate -cp template_newbox2.gro -cs spc216.gro -o template_solv2.gro -p ../topol_orig.top

# WARNING: Masses and atomic (Van der Waals) radii will be guessed
#          based on residue and atom names, since they could not be
#          definitively assigned from the information in your input
#          files. These guessed numbers might deviate from the mass
#          and radius of the atom type. Please check the output
#          files if necessary. Note, that this functionality may
#          be removed in a future GROMACS version. Please, consider
#          using another file format for your input.

# NOTE: From version 5.0 gmx solvate uses the Van der Waals radii
# from the source below. This means the results may be different
# compared to previous GROMACS versions.

# ++++ PLEASE READ AND CITE THE FOLLOWING REFERENCE ++++
# A. Bondi
# van der Waals Volumes and Radii
# J. Phys. Chem. 68 (1964) pp. 441-451

In [2]:
# adding NaCl and KCl to the box

# am confused about the mdp file used. the file uses 1nm as the cutoff distances for WDW/coulomb interactions
# while the cutoff used in actual simulation is 1.2
gmx_mpi grompp -f ../mdp_files/ions.mdp -c template_solv.gro -p ../topol.top -o genion_NaCl_input.tpr
gmx_mpi genion -s genion_NaCl_input.tpr -o genion_NaCl_output.gro -conc 0.137 -pname NA -nname CL -p ../topol.top

gmx_mpi grompp -f ../mdp_files/ions.mdp -c genion_NaCl_output.gro -p ../topol.top -o genion_KCl_input.tpr
gmx_mpi genion -s genion_KCl_input.tpr -o genion_KCl_output.gro -conc 0.00268 -pname K -nname CL -p ../topol.top

In [4]:
# energy minimization
# same confusion as previous cell regarding mdp file, cutoff distances and other parameters different
gmx_mpi grompp -f ../mdp_files/em.mdp -c genion_KCl_output.gro -p ../topol.top -o em_input.tpr -maxwarn 1
gerun gmx_mpi mdrun -s em_input.tpr -o em.trr -x em.xtc -c em_output.gro -g em.log -e em.edr -v -maxh 10

# Steepest Descents converged to Fmax < 1000 in 101 steps
# Potential Energy  = -2.6635816e+05
# Maximum force     =  9.3316010e+02 on atom 409
# Norm of force     =  7.3510789e+01

In [ ]:
# preliminary 2-ns simulation at 600 K in the NVT ensemble
# the paper doesn't mention position restraints. so no position restraints? obv no posres cuz want to find max rms replicas
# also which thermostat to use?
gmx_mpi grompp -f ../mdp_files/nvt-run-600.mdp -c em_output.gro -p ../topol.top -o nvt_600_input.tpr -maxwarn 1
gerun gmx_mpi mdrun -s nvt_600_input.tpr -o nvt_600.trr -x nvt_600.xtc -c nvt_600_output.gro -g nvt_600.log -e nvt.edr -v &> nvt_600.log

In [2]:
# algorithm to extract 48 confirmations

import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import math
from numpy import log2, zeros, mean, var, sum, loadtxt, arange, \
                  array, cumsum, dot, transpose, diagonal, floor
from numpy.linalg import inv, lstsq
import seaborn as sns
import scipy.cluster.hierarchy
from scipy.spatial.distance import squareform
import sklearn
from sklearn.decomposition import PCA
import itertools
import mdtraj as md


In [9]:
topology=f"em_output.gro"
trajectory=f"nvt_600.trr"
    
# print("pdb:",topology)
# print("trjaectory:",trajectory)

In [10]:
trj = md.load(trajectory, top=topology,stride=1)
trj = trj.atom_slice(range(627))
trj.center_coordinates()
top = trj.topology
first_frame = 0
last_frame = trj.n_frames
n_frames=trj.n_frames

n_frames

2001

In [21]:
trj2 = trj.slice(range(0, 2001, 10))
print(trj2.n_frames)

201


In [11]:
nres=[]



for res in trj.topology.residues: nres.append(res.resSeq)
sequence=(' %s' % [residue for residue in trj.topology.residues])
resname=(' %s' % [residue.name for residue in trj.topology.residues])
resindex=(' %s' % [residue.index for residue in trj.topology.residues])
res_num=(' %s' % [residue.resSeq  for residue in trj.topology.residues])



#log = open("/Users/paulrobustelli/Desktop/Sa_calc.log", "w")
print("** SYSTEM INFO **\n")
print("Number of atoms: %d\n" % trj.n_atoms)
print("Number of residues: %d\n" % len(set(nres)))
print("Number of frames: %d\n" % trj.n_frames)
print("Starting frame: %d\n" % first_frame)
print("Last frame: %d\n" % last_frame)
print("sequence: %s\n" % sequence)
print("residue names: %s\n" % resname)
print("residue number: %s\n" % res_num)
print("residue index: %s\n" % resindex)


** SYSTEM INFO **

Number of atoms: 627

Number of residues: 42

Number of frames: 2001

Starting frame: 0

Last frame: 2001

sequence:  [ASP1, ALA2, GLU3, PHE4, ARG5, HIS6, ASP7, SER8, GLY9, TYR10, GLU11, VAL12, HIS13, HIS14, GLN15, LYS16, LEU17, VAL18, PHE19, PHE20, ALA21, GLU22, ASP23, VAL24, GLY25, SER26, ASN27, LYS28, GLY29, ALA30, ILE31, ILE32, GLY33, LEU34, MET35, VAL36, GLY37, GLY38, VAL39, VAL40, ILE41, ALA42]

residue names:  ['ASP', 'ALA', 'GLU', 'PHE', 'ARG', 'HIS', 'ASP', 'SER', 'GLY', 'TYR', 'GLU', 'VAL', 'HIS', 'HIS', 'GLN', 'LYS', 'LEU', 'VAL', 'PHE', 'PHE', 'ALA', 'GLU', 'ASP', 'VAL', 'GLY', 'SER', 'ASN', 'LYS', 'GLY', 'ALA', 'ILE', 'ILE', 'GLY', 'LEU', 'MET', 'VAL', 'GLY', 'GLY', 'VAL', 'VAL', 'ILE', 'ALA']

residue number:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42]

residue index:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,

In [24]:
rmsd = md.rmsd(trj2, trj2, 0)
# print(max(rmsd))
rmsd = md.rmsd(trj2, trj2, 0)

# Find the index of the maximum value
max_index = np.argmax(rmsd)
max_index

148

In [27]:
# https://stackoverflow.com/questions/63321109/algorithm-for-selecting-a-subset-of-most-distant-elements
def find_largest_values(N, M):
    candidates = []  # List to store pairs (j, l(j))
    S_prime = []  # Resulting subset

    # Calculate pairwise distances and initialize candidates list
    for i in range(N):
            rmsd = md.rmsd(trj, trj, i)
            rmsd = np.delete(rmsd, i)

            l_i = min(rmsd)
            candidates.append((i, l_i))

    # Sort candidates list by l(i) in descending order
    candidates.sort(key=lambda x: x[1], reverse=True)

    # Select M elements with the largest values of l(i)
    for i in range(M):
        current_index, current_l_value = candidates[i]
        S_prime.append(current_index)

    return S_prime

indxs = find_largest_values(trj.n_frames, 48)
indxs

[1486,
 1209,
 1208,
 1220,
 1182,
 1197,
 1471,
 1476,
 1168,
 1244,
 1059,
 1483,
 1484,
 1485,
 1183,
 1184,
 1135,
 1185,
 1217,
 1352,
 931,
 1136,
 1137,
 837,
 1203,
 1219,
 1249,
 1468,
 1148,
 1475,
 1243,
 836,
 1202,
 1487,
 1488,
 1428,
 1437,
 1207,
 1865,
 1460,
 1869,
 1870,
 1149,
 1469,
 1470,
 1090,
 1196,
 1198]

In [ ]:
# NVT and NPT equilibration with position restraints on heavy atoms
